# Download and convert to Geopackage

This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Geopackage Database for future use.
Geopackage is a convenient data storage format, based on SQL, and is compatible with QGIS.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [ ]:
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [ ]:
# reading the area of interest polygon (could have been set)
gdf_geom = gpd.read_file('/home/hysope2/STUDIES/SWOT_Panama/DATA/aoi.gpkg')

# Limiting time period
dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)


## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [ ]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=0,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a geopackage database.
This geopackage format is quite efficient (though not the most efficient), and may easily be visualized in, e.g., QGIS
We are using the same geodataframe to limit the data to the area of interest

In [ ]:
from pixcdust.converters.gpkg import PixCNc2GpkgConverter
from glob import glob

In [ ]:
pixc = PixCNc2GpkgConverter(
            sorted(glob(pixcdownloader.path_download+'/*/*nc')),
            "/tmp/my_awesome_pixc_gpkg.gkpkg",
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

database has been succesfully created, we can remove the raw files

In [ ]:
import shutil
shutil.rmtree('/tmp/pixc')

# Read the database
previous steps are not necessary

Now we can open this database in a GeoDataFrame, load it in, e.g., QGIS, etc.

In [ ]:
from pixcdust.readers.gpkg import PixCGpkgReader

# nb: you may specify 
pixc_read = PixCGpkgReader(
    "/tmp/my_awesome_pixc_gpkg.gpkg"
)
pixc_read.read()
pixc_read.data

Enjoy!